In [1]:
import pandas as pd
import numpy as np
import sys

#Pfad zur PowerFectory Datei
#Markus:
#sys.path.append(r'C:\Program Files\DIgSILENT\PowerFactory 2024\Python\3.11')
#Johannes:
sys.path.append(r'D:\STUFF\PowerFactory\Python\3.11')
if __name__ == "__main__":
    import powerfactory as pf
app = pf.GetApplication()
if app is None:
    raise Exception('getting Powerfactory application failed')

In [2]:
#define project name and study case    
projName = 'BESS GFL-GFM_r03(19)'
study_case = '00_Comparison.IntCase'

#activate project
project = app.ActivateProject(projName)
proj = app.GetActiveProject()

#get the study case folder and activate project
oFolder_studycase = app.GetProjectFolder('study')
oCase = oFolder_studycase.GetContents(study_case)[0]
oCase.Activate()

#get load flow object and execute
oLoadflow=app.GetFromStudyCase('ComLdf') #get load flow object
oLoadflow.Execute() #execute load flow

0

#### function declaration

In [3]:
def createDf():
    columns = []
    Buses=app.GetCalcRelevantObjects('*.ElmTerm')
    i = 0
    for bus in Buses: #loop through list
        i += 1
        columns.append(getattr(bus, 'loc_name') + "Umag")
    for bus in Buses: #loop through list
        i += 1
        columns.append(getattr(bus, 'loc_name') + "Uang")
    for bus in Buses: #loop through list
        i += 1
        columns.append(getattr(bus, 'loc_name') + "Pact")
    for bus in Buses: #loop through list
        i += 1
        columns.append(getattr(bus, 'loc_name') + "Prea")
    print(columns)
    tup = (columns, [])
    df = pd.DataFrame(tup)
    df.columns = columns
    df = df.drop([0,1])
    return df

def measure(dfResults):
    dfResults.loc[len(dfResults)] = pd.Series(dtype='float64')
    Buses=app.GetCalcRelevantObjects('*.ElmTerm')
    for bus in Buses: #loop through list
        dfResults.at[dfResults.shape[0]-1, getattr(bus, 'loc_name') + "Umag"] = getattr(bus, 'm:ui')#measure Umag here
        
        dfResults.at[dfResults.shape[0]-1, getattr(bus, 'loc_name') + "Uang"] = getattr(bus, 'm:phiu')#measure Uang here
        
        dfResults.at[dfResults.shape[0]-1, getattr(bus, 'loc_name') + "Pact"] = getattr(bus, 'm:Pflow')#measure Pact here
        
        dfResults.at[dfResults.shape[0]-1, getattr(bus, 'loc_name') + "Prea"] = getattr(bus, 'm:Qflow')#measure Prea here

def splitter(dfResults, text):
    dfResults.loc[len(dfResults)] = pd.Series(dtype='float64')
    dfResults.at[dfResults.shape[0]-1,0] = text


#### setup (get vars and restore initial values)

In [4]:
#initial setup
Generators = app.GetCalcRelevantObjects('*.ElmSym')
for gen in Generators: #loop through list
    match getattr(gen, 'loc_name'):
        case 'Diesel Generator 1':
            DGen1 = gen
        case 'Diesel Generator 2':
            DGen2 = gen

Buses=app.GetCalcRelevantObjects('*.ElmTerm')
for bus in Buses: #loop through list
    match getattr(bus, 'loc_name'):
        case 'T7':
            Bus7 = bus
        case 'T8':
            Bus8 = bus


Loads=app.GetCalcRelevantObjects('*.ElmLod')
for load in Loads: #loop through list   
    match getattr(load, 'loc_name'):
        case 'Load_3':
            Load3 = load
        case 'Load_4':
            Load4 = load
        case 'Load_7':
            Load7 = load
        case 'Load_8':
            Load8 = load

winds = app.GetCalcRelevantObjects('*.ElmGenstat')
for wind in winds:
    match getattr(wind, 'loc_name'):
        case 'Wind1':
            Wind1 = wind
        case 'Wind2':
            Wind2 = wind
        case 'Speicher':
            Speicher = wind

PVs = app.GetCalcRelevantObjects('*.ElmPvsys')
for pv in PVs:
    match getattr(pv, 'loc_name'):
        case 'PV-Anlage':
            PV1 = pv
        

'''
Wind1.ELmGenstat
Wind2.ELmGenstat
PV-Anlage.ELmPvsys
Speicher.ELmGenstat

'''


setattr(DGen1, 'outserv', 0)
setattr(Wind1, 'outserv', 1)
setattr(Wind2, 'outserv', 1)
setattr(Speicher, 'outserv', 1)
setattr(PV1, 'outserv', 1)
setattr(Load3, 'plini', 0.55)
setattr(Load4, 'plini', 0.445)    
setattr(Load7, 'plini', 0.605)  
setattr(Load8, 'plini', 0.675)  
dfResults = createDf()
 



['T0Umag', 'T1Umag', 'T10Umag', 'T11Umag', 'T2Umag', 'T3Umag', 'T4Umag', 'T5Umag', 'T6Umag', 'T7Umag', 'T8Umag', 'T9Umag', 'Term Conv 1Umag', 'Term Conv 2Umag', 'T0Umag', 'T1Umag', 'T10Umag', 'T11Umag', 'T2Umag', 'T3Umag', 'T4Umag', 'T5Umag', 'T6Umag', 'T7Umag', 'T8Umag', 'T9Umag', 'Term Conv 1Umag', 'Term Conv 2Umag', 'T0Uang', 'T1Uang', 'T10Uang', 'T11Uang', 'T2Uang', 'T3Uang', 'T4Uang', 'T5Uang', 'T6Uang', 'T7Uang', 'T8Uang', 'T9Uang', 'Term Conv 1Uang', 'Term Conv 2Uang', 'T0Uang', 'T1Uang', 'T10Uang', 'T11Uang', 'T2Uang', 'T3Uang', 'T4Uang', 'T5Uang', 'T6Uang', 'T7Uang', 'T8Uang', 'T9Uang', 'Term Conv 1Uang', 'Term Conv 2Uang', 'T0Pact', 'T1Pact', 'T10Pact', 'T11Pact', 'T2Pact', 'T3Pact', 'T4Pact', 'T5Pact', 'T6Pact', 'T7Pact', 'T8Pact', 'T9Pact', 'Term Conv 1Pact', 'Term Conv 2Pact', 'T0Pact', 'T1Pact', 'T10Pact', 'T11Pact', 'T2Pact', 'T3Pact', 'T4Pact', 'T5Pact', 'T6Pact', 'T7Pact', 'T8Pact', 'T9Pact', 'Term Conv 1Pact', 'Term Conv 2Pact', 'T0Prea', 'T1Prea', 'T10Prea', 'T11Prea

## Aufgabe 1

In [5]:
oLoadflow=app.GetFromStudyCase('ComLdf') #get load flow object
oLoadflow.Execute() #execute load flow
measure(dfResults) 
splitter(dfResults,"Task 1 finsihed")

## Aufgabe 2

In [6]:
for i in range(1,11):
    setattr(Load3, 'plini', 0.55 + i*0.0045)
    
    oLoadflow=app.GetFromStudyCase('ComLdf') #get load flow object
    oLoadflow.Execute() #execute load flow
    measure(dfResults) 
    #ERGEBNISSE HIER AUSLESEN
    
    setattr(Load4, 'plini', 0.445 + i*0.0045)  

    oLoadflow.Execute() #execute load flow
    measure(dfResults) 
    #ERGEBNISSE HIER AUSLESEN

splitter(dfResults,"Task 1 finsihed")
#Switch off Diesel Generator, Test in Power Factory
#conditions: Load 3 active Power to 0.595/load4 active power to 0.49
splitter(dfResults, "Task 2, load 3 and 4 at max"  )

setattr(DGen2, 'outserv', 1)
oLoadflow.Execute() #execute load flow
measure(dfResults)
splitter(dfResults, "Gen 2 out of service"  )

setattr(DGen2, 'outserv', 0)
setattr(DGen1, 'outserv', 1)

for i in range(1,6):
    setattr(Load7, 'plini', 0.605 + i*0.0045)
    
    oLoadflow.Execute() #execute load flow
    measure(dfResults) 
    #ERGEBNISSE HIER AUSLESEN
    
    setattr(Load8, 'plini', 0.675 + i*0.0045)  

    oLoadflow.Execute() #execute load flow
    measure(dfResults) 
    #ERGEBNISSE HIER AUSLESEN


## Aufgabe 3

In [7]:
setattr(DGen2, 'outserv', 1)
setattr(PV1, 'outserv', 1)
setattr(Speicher, 'outserv', 1)



In [8]:
dfResults.to_excel("results.xlsx")

In [9]:
# calculate initial conditions
oInit = app.GetFromStudyCase('ComInc') #get initial condition calculation object
oInit.Execute()

#run RMS-simulation
oRms = app.GetFromStudyCase('ComSim') #get RMS-simulation object
oRms.Execute()

0

In [10]:
import numpy as np
import pandas as pd

def getResults():    
    #get result file
    elmRes = app.GetFromStudyCase('*.ElmRes')    
    app.ResLoadData(elmRes)
  
    #Get number of rows and columns
    NrRow = app.ResGetValueCount(elmRes,0)
    
    #get objects of interest
    oSDG1 = app.GetCalcRelevantObjects('Diesel Generator 1.ElmSym')[0]
    oSDG2 = app.GetCalcRelevantObjects('Diesel Generator 2.ElmSym')[0]
    oBus1 = app.GetCalcRelevantObjects('T1.ElmTerm')[0]
    oLine3_4 = app.GetCalcRelevantObjects('L_3_4.ElmLne')[0]

    #Get index of variable of interest
    ColIndex_time = app.ResGetIndex(elmRes,elmRes,'b:tnow')
    ColIndex_ut = app.ResGetIndex(elmRes,oSDG1,'s:ut')
    ColIndex_P = app.ResGetIndex(elmRes,oSDG1,'s:P1')
    ColIndex_Q = app.ResGetIndex(elmRes,oSDG1,'s:Q1')
    ColIndex_speed = app.ResGetIndex(elmRes,oSDG1,'s:xspeed')
    ColIndex_u_bus1 = app.ResGetIndex(elmRes,oBus1,'m:u')
    ColIndex_loading_line_3_4 = app.ResGetIndex(elmRes,oLine3_4,'c:loading')
    
    #pre-allocate result variables
    result_time = np.zeros((NrRow,))
    result_ut = np.zeros((NrRow))
    result_P = np.zeros((NrRow))
    result_Q = np.zeros((NrRow))    
    result_speed = np.zeros((NrRow))    
    result_u_bus1 = np.zeros((NrRow))    
    result_loading_line_3_4 = np.zeros((NrRow))    
    
    #get results for each time step
    for i in range(NrRow):    
        result_time[i] = app.ResGetData(elmRes,i,ColIndex_time)[1]
        result_ut[i] = app.ResGetData(elmRes,i,ColIndex_ut)[1]
        result_P[i] = app.ResGetData(elmRes,i,ColIndex_P)[1]
        result_Q[i] = app.ResGetData(elmRes,i,ColIndex_Q)[1]       
        result_speed[i] = app.ResGetData(elmRes,i,ColIndex_speed)[1]       
        result_u_bus1[i] = app.ResGetData(elmRes,i,ColIndex_u_bus1)[1]       
        result_loading_line_3_4[i] = app.ResGetData(elmRes,i,ColIndex_loading_line_3_4)[1]       
    
    results = pd.DataFrame()
    results['time'] = result_time
    results['P'] = result_P
    results['Q'] = result_Q
    results['ut'] = result_ut
    results['speed'] = result_speed
    results['u_bus1'] = result_u_bus1
    results['loading_line_3_4'] = result_loading_line_3_4
    return results

#query results
RES = getResults()

In [11]:
RES[0:20]

,time,P,Q,ut,speed,u_bus1,loading_line_3_4
0,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000
1,-0.9990,-0.9990,-0.9990,-0.9990,-0.9990,-0.9990,-0.9990
2,-0.9980,-0.9980,-0.9980,-0.9980,-0.9980,-0.9980,-0.9980
3,-0.9970,-0.9970,-0.9970,-0.9970,-0.9970,-0.9970,-0.9970
4,-0.9960,-0.9960,-0.9960,-0.9960,-0.9960,-0.9960,-0.9960
5,-0.9950,-0.9950,-0.9950,-0.9950,-0.9950,-0.9950,-0.9950
6,-0.9940,-0.9940,-0.9940,-0.9940,-0.9940,-0.9940,-0.9940
7,-0.9930,-0.9930,-0.9930,-0.9930,-0.9930,-0.9930,-0.9930
8,-0.9920,-0.9920,-0.9920,-0.9920,-0.9920,-0.9920,-0.9920
9,-0.9910,-0.9910,-0.9910,-0.9910,-0.9910,-0.9910,-0.9910
